In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

# sys.path.append("../response/")
# from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [14]:
corpus_path = "../../corpus/jsnli_1.1/"
data_name = "dev.tsv"

- data
['train_w_filtering.tsv', 'README.md', 'train_wo_filtering.tsv', 'dev.tsv']

In [15]:
df = pd.read_csv(corpus_path+data_name, delimiter='\t')

In [16]:
df

,label,pre,hypo
0,contradiction,家 の 前 の 雪 の 山 の 前 に 立って いる 数 人 の 人 と 、 シャベル を ...,マイアミ の 暑い 夏 の 日 です 。
1,contradiction,人 が ２ 頭 の 馬 の 間 に ひざまずいて いる,２ 頭 の 牛 の 間 に 人 が 立って い ます 。
2,contradiction,人 は 森 の 中 の 崖 に 登り 、 他 の 人 は 見て い ます 。,人 が 森 の 中 の 木 に 登る 。
3,contradiction,男 と 女 が 丘 の 上 を 歩き ます 。,男 と 女 が 座って い ます 。
4,contradiction,３ 人 、 １ 人 は 床 に 座って ギター を 弾き 、 ２ 人 は ソファ で ゲーム...,数 人 が ビーチ に 座って い ます 。
...,...,...,...
3911,neutral,バセットハウンド は 、 男 と 女 の 前 の 路地 の 戸口 に 縛ら れて い ます 。,それ は 彼 ら の 犬 です 。
3912,neutral,ローラーダービー の 前 に ストレッチ 。,競争 の 前 に ストレッチ を して いる 人 も い ます 。
3913,neutral,アクロバティック な 動き を 実行 する ステージ 上 の ２ 人 の 芸能 人 。,２ 人 の 芸能 人 が 異常に 大勢 の 観客 の ため に アクロバット を 行って い...
3914,neutral,バスケットボール コート の ２ 人 の 男性 が １ 人 は 白 で 、 もう １ 人 は...,男性 は バスケットボール トーナメント で プレー して い ます 。


In [19]:
df['label'].value_counts()

entailment       1432
neutral          1328
contradiction    1156
Name: label, dtype: int64

In [20]:
new_name = "dev_sentence.csv"


In [28]:
for i, (p, h) in  enumerate(zip( df.pre, df.hypo )) :
    p_ = p.replace(" ", "")
    df.iat[i, 1] = p_
    df.iat[i, 2] = h.replace(" ", "")

In [29]:
df

,label,pre,hypo
0,contradiction,家の前の雪の山の前に立っている数人の人と、シャベルを使っている人。,マイアミの暑い夏の日です。
1,contradiction,人が２頭の馬の間にひざまずいている,２頭の牛の間に人が立っています。
2,contradiction,人は森の中の崖に登り、他の人は見ています。,人が森の中の木に登る。
3,contradiction,男と女が丘の上を歩きます。,男と女が座っています。
4,contradiction,３人、１人は床に座ってギターを弾き、２人はソファでゲームをします。,数人がビーチに座っています。
...,...,...,...
3911,neutral,バセットハウンドは、男と女の前の路地の戸口に縛られています。,それは彼らの犬です。
3912,neutral,ローラーダービーの前にストレッチ。,競争の前にストレッチをしている人もいます。
3913,neutral,アクロバティックな動きを実行するステージ上の２人の芸能人。,２人の芸能人が異常に大勢の観客のためにアクロバットを行っています。
3914,neutral,バスケットボールコートの２人の男性が１人は白で、もう１人は青で、白の選手はチームメイトが見な...,男性はバスケットボールトーナメントでプレーしています。


In [30]:
df.to_csv(corpus_path+new_name, index=False, encoding="UTF-8")